<a href="https://colab.research.google.com/github/durfred/my-first-binder/blob/main/picoGPT_cuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 确保你使用的是GPU运行时 (Runtime -> Change runtime type -> GPU)
# 安装所有需要的库
!pip install torch transformers numpy cupy-cuda12x requests tqdm -q # -q 静默安装

# 导入所有需要的模块
import cupy as np      # CuPy 将作为 np
import numpy as np_cpu # NumPy 将作为 np_cpu，以避免与 CuPy 混淆
import torch           # 用于 Hugging Face 模型
import os              # 用于文件操作
from tqdm.autonotebook import tqdm # 进度条显示
from transformers import AutoTokenizer, GPT2LMHeadModel, GPT2Config # Hugging Face 相关

print("所有库安装完成。")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 42.9 MB/s eta 0:00:00


<ipython-input-1-564246711>:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm # 进度条显示


所有库安装完成。


In [2]:
# --- picoGPT 核心函数 (已修改为 CuPy 并修复了所有已知问题) ---

def gpt2(inputs, wte, wpe, blocks, ln_f_g, ln_f_b, n_head):
    """
    GPT-2 模型的前向传播。所有输入权重都必须是 CuPy 数组。
    inputs: (n_seq) - 当前输入token的序列ID
    wte: (n_vocab, n_embd) - token嵌入权重
    wpe: (n_seq_max, n_embd) - 位置嵌入权重 (n_seq_max 是 block_size)
    blocks: 包含所有Transformer块参数的列表
    ln_f_g, ln_f_b: 最终层归一化的增益和偏置
    n_head: 注意力头的数量
    """
    n_seq = inputs.shape[0] # 当前输入序列的长度
    n_embd = wte.shape[1]   # 嵌入维度

    # 1. token + positional embeddings (令牌嵌入 + 位置嵌入)
    # wte[inputs] 进行token查找，wpe[:n_seq] 获取当前序列长度对应的位置嵌入
    x = wte[inputs] + wpe[:n_seq]

    # 2. Transformer blocks (Transformer 块)
    for i in range(len(blocks)):
        # block函数需要解包blocks[i]中的所有参数
        x = block(x, *blocks[i], n_head)

    # 3. Final layer normalization (最终层归一化)
    x = layernorm(x, ln_f_g, ln_f_b)

    # 4. Linear layer (output logits) (线性层，输出logits)
    # 将结果乘以 token 嵌入矩阵的转置，得到每个词的概率分数
    logits = x @ wte.T

    return logits

def block(x, ln_1_g, ln_1_b, attn_c_attn_w, attn_c_attn_b, attn_c_proj_w, attn_c_proj_b, mlp_c_fc_w, mlp_c_fc_b, mlp_c_proj_w, mlp_c_proj_b, n_head):
    """一个 Transformer 块的前向传播"""
    # 1. 第一个层归一化和注意力机制
    attn_output = attn(layernorm(x, ln_1_g, ln_1_b), attn_c_attn_w, attn_c_attn_b, attn_c_proj_w, attn_c_proj_b, n_head)
    x = x + attn_output # 残差连接

    # 2. 第二个层归一化和MLP (多层感知机)
    # 注意：这里的layernorm再次使用ln_1_g,ln_1_b，与原始picoGPT代码一致
    mlp_output = mlp(layernorm(x, ln_1_g, ln_1_b), mlp_c_fc_w, mlp_c_fc_b, mlp_c_proj_w, mlp_c_proj_b)
    x = x + mlp_output # 残差连接

    return x

def layernorm(x, g, b):
    """层归一化"""
    mean = np.mean(x, axis=-1, keepdims=True)
    variance = np.mean(np.square(x - mean), axis=-1, keepdims=True)
    # 加上一个很小的epsilon以防止除以零
    x = (x - mean) * (1 / np.sqrt(variance + 1e-5)) * g + b
    return x

def attn(x, c_attn_w, c_attn_b, c_proj_w, c_proj_b, n_head):
    """自注意力机制"""
    n_seq, n_embd = x.shape
    n_state = n_embd // n_head # 每个注意力头的维度

    # --- 请在此处添加以下调试打印语句 ---
    print(f"DEBUG (attn): 输入 x 的形状: {x.shape}")
    print(f"DEBUG (attn): c_attn_w 的形状: {c_attn_w.shape}")
    print(f"DEBUG (attn): n_embd (从 x 推断): {n_embd}")
    print(f"DEBUG (attn): n_head: {n_head}")
    # --- 调试打印语句结束 ---

    # 1. 查询 (Q), 键 (K), 值 (V) 投影
    x = x @ c_attn_w + c_attn_b
    q, k, v = np.split(x, 3, axis=-1) # 将输出沿最后一个维度分成3份 (Q, K, V)

    # 2. 分割成多个注意力头
    # (n_seq, n_head, n_state) -> transpose (n_head, n_seq, n_state)
    q = q.reshape(n_seq, n_head, n_state).transpose(1, 0, 2)
    k = k.reshape(n_seq, n_head, n_state).transpose(1, 0, 2)
    v = v.reshape(n_seq, n_head, n_state).transpose(1, 0, 2)

    # 3. 计算注意力分数 (Q @ K^T) / sqrt(d_k) + 因果掩码 + Softmax
    scores = q @ k.transpose(0, 2, 1) / np.sqrt(n_state)

    # 创建因果掩码 (上三角矩阵为负无穷，防止看到未来信息)
    causal_mask = (1 - np.tri(n_seq, dtype=np.float32)) * -1e10
    scores = scores + causal_mask # 应用掩码

    # Softmax 归一化
    scores = np.exp(scores - np.max(scores, axis=-1, keepdims=True))
    scores = scores / np.sum(scores, axis=-1, keepdims=True)

    # 4. 加权求和 (scores @ V)
    x = scores @ v
    # 拼接多个注意力头的输出
    x = x.transpose(1, 0, 2).reshape(n_seq, n_embd)

    # 5. 线性投影
    x = x @ c_proj_w + c_proj_b

    return x

def mlp(x, c_fc_w, c_fc_b, c_proj_w, c_proj_b):
    """多层感知机 (MLP)"""
    # 1. 线性层 1
    x = x @ c_fc_w + c_fc_b

    # 2. GELU 激活函数
    # 使用近似公式 0.5 * x * (1 + tanh(sqrt(2/pi) * (x + 0.044715 * x^3)))
    x = 0.5 * x * (1 + np.tanh(np.sqrt(2 / np.pi) * (x + 0.044715 * np.power(x, 3))))

    # 3. 线性层 2
    x = x @ c_proj_w + c_proj_b

    return x

print("picoGPT核心函数定义完成。")

picoGPT核心函数定义完成。


In [3]:
# --- Hugging Face GPT-2 权重转换脚本 (已修正转置问题) ---

def convert_hf_to_pico_weights(model_name="gpt2", output_filename="model.npz"):
    """
    从 Hugging Face 加载 GPT-2 权重，并将其转换为 picoGPT 的 .npz 格式。
    已修正 Conv1D 权重的转置问题。
    """
    print(f"1. 加载 Hugging Face GPT-2 模型: {model_name}...")
    model = GPT2LMHeadModel.from_pretrained(model_name)
    state_dict = model.state_dict()

    pico_weights = {}

    print("2. 转换 Embedding (词嵌入) 和最终层归一化权重...")
    # Token Embedding
    pico_weights['wte'] = state_dict['transformer.wte.weight'].numpy()
    # Positional Embedding
    pico_weights['wpe'] = state_dict['transformer.wpe.weight'].numpy()

    # Final Layer Normalization (最终层归一化)
    pico_weights['ln_f_g'] = state_dict['transformer.ln_f.weight'].numpy()
    pico_weights['ln_f_b'] = state_dict['transformer.ln_f.bias'].numpy()

    print(f"3. 转换 {model.config.n_layer} 个 Transformer 块的权重...")
    for i in range(model.config.n_layer):
        prefix = f'transformer.h.{i}.'
        block_key = f'h{i}'

        # Layer Normalization 1
        pico_weights[f'{block_key}_ln_1_g'] = state_dict[prefix + 'ln_1.weight'].numpy()
        pico_weights[f'{block_key}_ln_1_b'] = state_dict[prefix + 'ln_1.bias'].numpy()

        # Attention weights (注意力权重) - 移除 .T
        # Hugging Face Conv1D 权重已经是 (in_features, out_features) 形状，不需要转置
        pico_weights[f'{block_key}_attn_c_attn_w'] = state_dict[prefix + 'attn.c_attn.weight'].numpy() # REMOVED .T
        pico_weights[f'{block_key}_attn_c_attn_b'] = state_dict[prefix + 'attn.c_attn.bias'].numpy()

        pico_weights[f'{block_key}_attn_c_proj_w'] = state_dict[prefix + 'attn.c_proj.weight'].numpy() # REMOVED .T
        pico_weights[f'{block_key}_attn_c_proj_b'] = state_dict[prefix + 'attn.c_proj.bias'].numpy()

        # MLP (前馈网络) 权重 - 移除 .T
        pico_weights[f'{block_key}_mlp_c_fc_w'] = state_dict[prefix + 'mlp.c_fc.weight'].numpy() # REMOVED .T
        pico_weights[f'{block_key}_mlp_c_fc_b'] = state_dict[prefix + 'mlp.c_fc.bias'].numpy()

        pico_weights[f'{block_key}_mlp_c_proj_w'] = state_dict[prefix + 'mlp.c_proj.weight'].numpy() # REMOVED .T
        pico_weights[f'{block_key}_mlp_c_proj_b'] = state_dict[prefix + 'mlp.c_proj.bias'].numpy()

    # 将转换后的权重保存为 .npz 格式
    np_cpu.savez(output_filename, pico_weights)
    print(f"4. 转换并保存权重到 {output_filename}. 文件大小: {os.path.getsize(output_filename)/(1024*1024):.2f} MB")

    return output_filename

# --- 执行权重转换 ---
!rm -f model.npz # 移除旧的 model.npz 文件
print("前一个 model.npz 文件已移除。")

# 重新执行转换过程，生成新的 model.npz 文件
generated_npz_file = convert_hf_to_pico_weights(model_name="gpt2", output_filename="model.npz")

print(f"\n成功创建了 {generated_npz_file}. 文件已准备好加载。")

前一个 model.npz 文件已移除。
1. 加载 Hugging Face GPT-2 模型: gpt2...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

2. 转换 Embedding (词嵌入) 和最终层归一化权重...
3. 转换 12 个 Transformer 块的权重...
4. 转换并保存权重到 model.npz. 文件大小: 474.64 MB

成功创建了 model.npz. 文件已准备好加载。


In [4]:
# --- 加载 picoGPT 权重 (已修改为 CuPy 并兼容扁平化结构) ---

def load_gpt2_weights_cupy(filename="model.npz"):
    """
    加载 GPT-2 small 模型权重，并将其加载为 CuPy 数组。
    兼容由 `convert_hf_to_pico_weights` 函数生成的扁平化 .npz 文件。
    """
    if not os.path.exists(filename):
        print(f"错误: {filename} 未找到。请确保 Cell 3 成功运行并生成了文件！")
        return None # 或者抛出异常

    print(f"加载 {filename} (由转换脚本生成)...")
    # 使用 NumPy (CPU) 加载 .npz 文件
    params_npz_file = np_cpu.load(filename, allow_pickle=True)

    # 获取包含所有权重的字典 (它被保存为 'arr_0' 键的值)
    params_dict = params_npz_file['arr_0'].item()

    # 提取GPT-2 small的固定配置参数
    n_vocab = 50257 # 词汇表大小
    n_embd = 768    # 嵌入维度
    n_head = 12     # 注意力头数量
    n_block = 12    # Transformer 块数量
    block_size = 1024 # 模型上下文窗口大小 (最大序列长度)

    # 映射权重到picoGPT的参数，并将其转换为 CuPy 数组 (加载到 GPU 内存)
    wte = np.asarray(params_dict['wte'])
    wpe = np.asarray(params_dict['wpe'])
    ln_f_g = np.asarray(params_dict['ln_f_g'])
    ln_f_b = np.asarray(params_dict['ln_f_b'])

    blocks = []
    for i in range(n_block):
        block_key = f'h{i}' # 构造扁平化键名 (例如 'h0', 'h1'...)

        # 从扁平化字典中直接获取权重，并转换为 CuPy 数组
        ln_1_g = np.asarray(params_dict[f'{block_key}_ln_1_g'])
        ln_1_b = np.asarray(params_dict[f'{block_key}_ln_1_b'])
        attn_c_attn_w = np.asarray(params_dict[f'{block_key}_attn_c_attn_w'])
        attn_c_attn_b = np.asarray(params_dict[f'{block_key}_attn_c_attn_b'])
        attn_c_proj_w = np.asarray(params_dict[f'{block_key}_attn_c_proj_w'])
        attn_c_proj_b = np.asarray(params_dict[f'{block_key}_attn_c_proj_b'])
        mlp_c_fc_w = np.asarray(params_dict[f'{block_key}_mlp_c_fc_w'])
        mlp_c_fc_b = np.asarray(params_dict[f'{block_key}_mlp_c_fc_b'])
        mlp_c_proj_w = np.asarray(params_dict[f'{block_key}_mlp_c_proj_w'])
        mlp_c_proj_b = np.asarray(params_dict[f'{block_key}_mlp_c_proj_b'])

        # 将当前块的所有参数打包成一个列表，添加到 blocks 列表中
        blocks.append([ln_1_g, ln_1_b, attn_c_attn_w, attn_c_attn_b, attn_c_proj_w, attn_c_proj_b,
                       mlp_c_fc_w, mlp_c_fc_b, mlp_c_proj_w, mlp_c_proj_b])

    print("GPT-2 small 权重已成功加载到 CuPy (GPU 内存)。")
    return wte, wpe, blocks, ln_f_g, ln_f_b, n_head, n_vocab, block_size

# --- 执行加载 ---
# 调用函数加载权重，并将返回的参数解包到对应变量
wte, wpe, blocks, ln_f_g, ln_f_b, n_head, n_vocab, block_size = load_gpt2_weights_cupy()

加载 model.npz (由转换脚本生成)...
GPT-2 small 权重已成功加载到 CuPy (GPU 内存)。


In [5]:
# --- 文本生成函数 (CuPy 版本) ---

def generate_text_cupy(prompt, model_params, tokenizer, max_tokens=50, temperature=1.0, top_k=0):
    """
    使用 CuPy 加载的 GPT-2 模型生成文本。
    prompt: 初始提示文本 (字符串)
    model_params: 包含所有模型参数的元组 (wte, wpe, blocks, ...)
    tokenizer: Hugging Face 分词器
    max_tokens: 最大生成token数量
    temperature: 采样温度 (越高越随机)
    top_k: Top-K 采样参数 (只考虑概率最高的K个token)
    """
    wte, wpe, blocks, ln_f_g, ln_f_b, n_head, n_vocab, block_size = model_params

    # 1. 编码提示 (分词器在 CPU 上工作)
    # return_tensors="np" 返回 NumPy 数组
    input_ids_np = tokenizer.encode(prompt, return_tensors="np")[0].astype(np_cpu.int32)

    # 确保输入长度不超过模型的最大上下文窗口
    if len(input_ids_np) > block_size:
        input_ids_np = input_ids_np[-block_size:] # 截断为最后 block_size 个 token

    # 关键：将初始输入 token ID 从 NumPy (CPU) 转移到 CuPy (GPU)
    input_ids_gpu = np.asarray(input_ids_np)

    # generated_ids 列表在 CPU 上维护，因为 tokenizer.decode 需要 NumPy 数组
    generated_ids = list(input_ids_np)

    print(f"正在生成 {max_tokens} 个 token (temperature={temperature}, top_k={top_k})...")
    print(f"提示: {prompt}")

    for _ in tqdm(range(max_tokens), desc="生成中"):
        # 2. 准备当前输入：取当前生成序列的最后 `block_size` 个 token
        # 每次迭代，将当前生成的序列 (或其截断部分) 从 CPU 移到 GPU
        current_input_gpu = np.asarray(np_cpu.array(generated_ids[-block_size:]))

        # 3. 运行 picoGPT 模型前向传播 (在 GPU 上执行)
        logits_gpu = gpt2(current_input_gpu, wte, wpe, blocks, ln_f_g, ln_f_b, n_head)

        # 4. 获取最后一个 token 的 logits (预测下一个词的概率分数)
        next_token_logits_gpu = logits_gpu[-1, :]

        # 5. 应用 temperature (温度) 和 Top-K 采样
        if temperature != 0.0:
            next_token_logits_gpu = next_token_logits_gpu / temperature

        if top_k > 0:
            # 对于 Top-K 采样，需要将 logits 移回 CPU 进行排序和过滤
            next_token_logits_cpu = next_token_logits_gpu.get() # .get() 将 CuPy 数组复制回 NumPy (CPU)
            # 将非 Top-K 的 logits 设置为负无穷，使其在 softmax 后概率为0
            indices_to_remove = next_token_logits_cpu < np_cpu.sort(next_token_logits_cpu)[-top_k]
            next_token_logits_cpu[indices_to_remove] = -float('Inf')
            # 重新将其转换为 CuPy 数组 (虽然这里直接采样了，但好的习惯)
            next_token_logits_gpu = np.asarray(next_token_logits_cpu)


        # 6. Softmax 转换为概率 (仍在 GPU 上)
        # 减去最大值以提高数值稳定性
        probabilities_gpu = np.exp(next_token_logits_gpu - np.max(next_token_logits_gpu))
        probabilities_gpu = probabilities_gpu / np.sum(probabilities_gpu)

        # 7. 采样下一个 token ID
        # 将概率数组从 GPU 移回 CPU，因为 np_cpu.random.choice 需要 NumPy 数组
        next_token_id = np_cpu.random.choice(len(probabilities_gpu), p=probabilities_gpu.get())

        # 8. 将新生成的 token ID 添加到已生成序列中
        generated_ids.append(int(next_token_id)) # 确保是 Python int 类型

    # 9. 解码整个生成序列 (分词器在 CPU 上解码)
    full_generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
    print("\n--- 完整生成文本 ---")
    print(full_generated_text)
    return full_generated_text

print("文本生成函数定义完成。")

文本生成函数定义完成。


In [6]:
# In your final Colab cell:
# --- Instantiate Tokenizer and Run Text Generation ---

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model_parameters_cupy = (wte, wpe, blocks, ln_f_g, ln_f_b, n_head, n_vocab, block_size)

prompt_cupy = "The quick brown fox jumps over the lazy"

print("\nStarting text generation...")
# *** EXPERIMENT WITH THESE VALUES ***
generated_output_cupy = generate_text_cupy(
    prompt_cupy,
    model_parameters_cupy,
    tokenizer,
    max_tokens=50,
    temperature=0.8, # Try increasing slightly from 0.7 to introduce more variance
    top_k=25         # Try significantly decreasing from 50 to focus on higher probability tokens
)

print("\n文本生成完成。")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]


Starting text generation...
正在生成 50 个 token (temperature=0.8, top_k=25)...
提示: The quick brown fox jumps over the lazy


生成中:   0%|          | 0/50 [00:00<?, ?it/s]

DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 2304)
DEBUG (attn): n_embd (从 x 推断): 768
DEBUG (attn): n_head: 12
DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 2304)
DEBUG (attn): n_embd (从 x 推断): 768
DEBUG (attn): n_head: 12
DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 2304)
DEBUG (attn): n_embd (从 x 推断): 768
DEBUG (attn): n_head: 12
DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 2304)
DEBUG (attn): n_embd (从 x 推断): 768
DEBUG (attn): n_head: 12
DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 2304)
DEBUG (attn): n_embd (从 x 推断): 768
DEBUG (attn): n_head: 12
DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 2304)
DEBUG (attn): n_embd (从 x 推断): 768
DEBUG (attn): n_head: 12
DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 2304)
DEBUG (attn): n_embd (从 x 推断): 768
DEBUG (attn): n_head: 12
DEBUG (attn): 输入 x 的形状: (8, 768)
DEBUG (attn): c_attn_w 的形状: (768, 23